In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
!nvidia-smi

Sat Oct  3 07:52:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setup conda environment to download RDKIt
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# update 1
!conda install -q -y --prefix /usr/local python=3.6 ujson

# update 2
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
# test it
import ujson
print(ujson.dumps({1:2}))

#RDKIT installation
!conda install -y -c rdkit rdkit

# Pytorch geometric
# Install required packages.
!pip install -q torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install -q torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

# Reverting back to previous version of torch 
# !pip install torch==1.0.0 torchvision==0.2.1

--2020-10-03 07:53:25--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   224MB/s    in 0.2s    

2020-10-03 07:53:25 (224 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6

In [ ]:
import pandas as pd
import numpy as np
import os
import re 
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt 
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_columns = 999
plt.rcParams['figure.figsize'] = (15, 6)

#size=25
size=15
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 25}
          
plt.rcParams.update(params)
import seaborn as sns
sns.set()

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

# from rdkit import Chem
# from rdkit.Chem import AllChem
# from rdkit.Chem.Draw import rdMolDraw2D

## `GPU Specs`

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

  Running setup.py bdist_wheel for gputil ... done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
    100% |████████████████████████████████| 460kB 16.9MB/s 
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/39/a0/f5/c4fa280463e29aea07797acb5312358fefb067c1f4f98e11b1
Successfully built psutil
    100% |████████████████████████████████| 71kB 8.3MB/s 


# `Load the Complete data and specify the relevant arguments in the graph_data_prep script to create the graph data succesfully`
- `Utility to create the graph data and it stores the node features, edge features and the labels in the directory specified by the user`

In [ ]:
from graph_data_prep import *
from graph_neural_networks import *

config = {'graph_data_path': 'path/to/dave/graph/data',
            'smile_data_path_train' : 'path/to/smile_data'}

complete_protein_data_2 = pd.read_csv(config['smile_data_path_train'])
dataset = MyOwnDataset(config['graph_data_path'])

  0%|          | 0/9113 [00:00<?, ?it/s]

Processing...


100%|██████████| 9113/9113 [34:15<00:00,  4.43it/s]

Done!


In [ ]:
dataset = dataset.shuffle()
one_tenth_length = int(len(dataset) * 0.1)

train_dataset = dataset[:one_tenth_length * 9]
val_dataset = dataset[one_tenth_length*9 : ]
# test_dataset = dataset[one_tenth_length*9:]

len(train_dataset), len(val_dataset)
# , len(test_dataset)

from torch_geometric.data import DataLoader
batch_size= 128
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader =   DataLoader(val_dataset, batch_size = batch_size, shuffle=True)
# test_loader =  DataLoader(test_dataset, batch_size = batch_size, shuffle=True)

loader_check = iter(train_loader).next()
node, edge = loader_check.x, loader_check.edge_index
node.shape, edge.min(), edge.max(), edge.shape

(torch.Size([167763, 14]),
 tensor(0, device='cuda:0'),
 tensor(167762, device='cuda:0'),
 torch.Size([2, 342380]))

# New datloader 

In [ ]:
import collections
def get_max_length(x):
    return len(max(x, key=len))

def pad_sequence(seq):
    def _pad(_it, _max_len):
        return [0] * (_max_len - len(_it)) + _it
    return [_pad(it, get_max_length(seq)) for it in seq]

def custom_collate(batch):
    transposed = zip(*batch)
    lst = []
    for samples in transposed:
        if isinstance(samples[0], int):
            lst.append(torch.LongTensor(samples))
        elif isinstance(samples[0], float):
            lst.append(torch.DoubleTensor(samples))
        elif isinstance(samples[0], collections.Sequence):
            lst.append(torch.LongTensor(pad_sequence(samples)))
    return lst            

# `Instantiating the train and test data loaders`

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test = train_test_split(complete_protein_data_2, test_size = 0.3)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

train_dataset = ProtData(X_train,  ring_dict, chiral_centers, hybridization_list, smile_col = 'smile_notation', target_col = 'target_label')
validation_data = ProtData(X_test, ring_dict, chiral_centers, hybridization_list,  smile_col = 'smile_notation', target_col = 'target_label')

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(validation_data, batch_size = 32, shuffle=True, collate_fn=custom_collate)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

# model = GCN(in_channels = in_channels,  hidden_channels = hidden_channels, output_dim = output_dim)
# model.to(device)

model = SortPool(in_channels,  hidden_channels, output_dim, num_layers = 3)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5)

<IPython.core.display.Javascript object>

In [ ]:
def train():
    epoch_loss = []
    model.train()

    loop = tqdm(enumerate(train_loader), position=0, leave=True)
    for i, data in loop:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         targets = torch.tensor(data.y, dtype= torch.long).to(device)

         loss = criterion(out, targets)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()
         epoch_loss.append(loss.item())

        #  # GPU memory usage information 
        #  GPUs = GPU.getGPUs()
        #  gpu = GPUs[0]
        #  process = psutil.Process(os.getpid())

         loop.set_description(f"Epoch {epoch +1}/{200}")
         loop.set_postfix(loss = loss.item(), mean_loss = np.mean(epoch_loss))

    # scheduler()
    scheduler.step(np.mean(epoch_loss))
    return np.mean(epoch_loss)

from sklearn.metrics import f1_score
def test(loader):
  model.eval()

  correct = 0
  loop = tqdm(enumerate(loader), position=0, leave=True)
  preds = []
  true = []

  with torch.no_grad():
    for i, data in loop:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim = 1)  # Use the class with highest probability.
        targets = torch.tensor(data.y, dtype= torch.long).to(device)

        preds += pred.to('cpu').numpy().tolist()
        true += targets.to('cpu').numpy().tolist()

        correct += int((pred == targets).sum())  # Check against ground-truth labels.
  
  f1 = f1_score(true, preds, average='weighted')
  return correct/len(loader.dataset), f1  # Derive ratio of correct predictions.

In [ ]:
losses = []
counter = 0
lr = 0.0001
for epoch in tqdm(range(0, 200), position = 0, leave = True):
    epoch_loss = train()
    losses.append(epoch_loss)

    if (epoch > 10) :
      if (np.round(losses[epoch], 2) >= np.round(losses[epoch - 1], 2)):
        counter += 1
    
    if (counter > 1 and counter%15 == 0):
      lr -= lr * 0.1
      print(f"Learning rate has been decreased to {lr}")
      for param_group in optimizer.param_groups:
        param_group['lr'] = lr
      counter = 0

    val_acc, val_f1 = test(val_loader)
    print(f'Epoch: {epoch:03d}, Val Acc: {val_acc:.4f} Val f1: {val_f1:.3f} counter : {counter}')

Epoch 1/200: : 65it [00:20,  3.21it/s, loss=1.28, mean_loss=1.33]
8it [00:01,  4.27it/s]
0it [00:00, ?it/s]

Epoch: 000, Val Acc: 0.3851 Val f1: 0.214 counter : 0


Epoch 2/200: : 65it [00:19,  3.26it/s, loss=1.21, mean_loss=1.32]
8it [00:02,  3.76it/s]
0it [00:00, ?it/s]

Epoch: 001, Val Acc: 0.3851 Val f1: 0.214 counter : 0


Epoch 3/200: : 65it [00:21,  3.02it/s, loss=1.56, mean_loss=1.33]
8it [00:01,  4.17it/s]
0it [00:00, ?it/s]

...

Epoch: 197, Val Acc: 0.4431 Val f1: 0.409 counter : 1


Epoch 199/200: : 65it [00:20,  3.15it/s, loss=1.19, mean_loss=1.03]
8it [00:01,  4.02it/s]
0it [00:00, ?it/s]

Epoch: 198, Val Acc: 0.4442 Val f1: 0.414 counter : 1


Epoch 200/200: : 65it [00:20,  3.18it/s, loss=0.802, mean_loss=1.03]
8it [00:01,  4.08it/s]
100%|██████████| 200/200 [1:11:06<00:00, 21.33s/it]

Epoch: 199, Val Acc: 0.4475 Val f1: 0.418 counter : 2


# `Model evaluation and saving the graph model`

In [ ]:
model.eval()
with torch.no_grad():
  for i, data in enumerate(val_loader):
    x, edge_index, batch = data.x, data.edge_index, data.batch
    output = model(x, edge_index, batch)
    targets = data.y
    print(output, targets)
    break

tensor([[ 2.2408,  1.6871,  0.6324,  ..., -2.4726, -1.2293, -0.9360],
        [ 3.2180,  2.0356,  0.7193,  ..., -2.4270, -0.9666, -1.3688],
        [ 1.8113,  1.7276,  0.2888,  ..., -2.6338, -1.1362, -1.5167],
        ...,
        [ 3.8880,  2.3889,  1.5504,  ..., -3.3505, -0.3370, -2.0742],
        [ 2.9465,  1.5810,  0.9186,  ..., -2.0710, -0.9514, -0.9937],
        [ 2.0554,  1.6099,  0.6297,  ..., -2.5760, -1.2069, -1.2641]],
       device='cuda:0') [0, 0, 12, 7, 0, 8, 14, 3, 1, 15, 0, 3, 0, 13, 0, 25, 22, 23, 2, 0, 0, 29, 0, 3, 22, 7, 1, 2, 0, 0, 0, 10, 1, 1, 3, 0, 11, 0, 0, 28, 3, 0, 3, 4, 0, 8, 9, 14, 14, 17, 1, 5, 6, 0, 2, 7, 0, 19, 35, 0, 0, 1, 0, 17, 0, 0, 9, 4, 0, 7, 0, 0, 0, 7, 3, 10, 8, 0, 17, 6, 32, 9, 2, 0, 12, 0, 1, 26, 7, 4, 0, 0, 0, 1, 0, 0, 0, 1, 5, 9, 0, 7, 3, 0, 3, 0, 21, 1, 0, 6, 6, 20, 0, 7, 0, 8, 12, 2, 0, 0, 9, 0, 1, 0, 0, 2, 0, 1]
